In [1]:
import importlib.util
import pandas as py
import numpy as np

In [2]:
spec = importlib.util.spec_from_file_location("DataPreprocessing", "/home/admin1/PycharmProjects/Machine_learning_Python/Utility/DataPreprocessing.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
ab = foo.DataPreprocessing()

Data PreProcessign object created successfully
